In [1]:
import os
import pathlib
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
from timeit import default_timer as timer

from LoadData import LoadData

import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics, svm
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.tree._tree import TREE_LEAF

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.utils import parallel_backend


TESTING = True
DECISION_TREE = False
SUPPORT_VECTOR = True
NEURAL_NET = False
K_NEAREST = False
BOOSTING = False
NORMALIZE_DATA = False
USE_PCA = True
DataSetName = "MNIST"


In [2]:
cwd = pathlib.Path().absolute()
if DataSetName == "MNIST":
    training_data_path = "{}/mnist-train-data.csv".format(cwd)
    testing_data_path = "{}/mnist-test-data.csv".format(cwd)
else:
    training_data_path = "{}/fashion-mnist-train-data.csv".format(cwd)
    testing_data_path = "{}/fashion-mnist-test-data.csv".format(cwd)

with parallel_backend('threading'):
    training_labels, training_data, _ = LoadData(training_data_path, normalize=NORMALIZE_DATA)
    testing_labels, testing_data, _ = LoadData(testing_data_path, normalize=NORMALIZE_DATA)

Scaler = StandardScaler().fit(training_data)
        
training_data = Scaler.transform(training_data)
testing_data = Scaler.transform(testing_data)


Attempting to load: mnist-train-data.csv

Loading Complete
Data Statistics: 
   Number of Entries: 60000 
   Shape of Entry: (785,)

Attempting to load: mnist-test-data.csv

Loading Complete
Data Statistics: 
   Number of Entries: 10000 
   Shape of Entry: (785,)



In [3]:
def FindCorrectPredictions(prediction_array, actual_array):
    idx = np.argwhere(actual_array != prediction_array)
    actual = actual_array[idx]
    predicted = prediction_array[idx]
    pred_actual_hstack = np.hstack((predicted, actual))
    final_hstack = np.hstack((idx, pred_actual_hstack))
    df = pd.DataFrame(final_hstack, columns=["Index Of Image", "Predicted Image", "Actual Image"])
    return df

In [4]:
def prune_index11(clf, index, threshold):
    try:
        """
        prune_index(dt.tree_, 0, 3)
        https://stackoverflow.com/questions/49428469/pruning-decision-trees
        """

        if clf.tree_.value[index][0].sum() < threshold:
            # turn node into a leaf by "unlinking" its children
            clf.tree_.children_left[index] = TREE_LEAF
            clf.tree_.children_right[index] = TREE_LEAF
        # if there are children, visit them as well
        if clf.tree_.children_left[index] != TREE_LEAF:
            prune_index(clf.tree_.children_left[index], threshold)
            prune_index(clf.tree_.children_right[index], threshold)

    except Exception as err:
        print("Exception occurred while pruning Decision Tree. \n", err)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)

In [ ]:
NeuralNetwork_Classifier = MLPClassifier(solver='sgd', max_iter=200, verbose=1, hidden_layer_sizes=(100,))

In [ ]:
KNN_Classifier = KNeighborsClassifier()

In [ ]:
Boosted_Classifier = GradientBoostingClassifier(n_estimators=100, max_depth=100, verbose=3)

In [ ]:
SVM_Classifier = svm.SVC(kernel='rbf', verbose=3, max_iter=1000)

In [ ]:
DecisionTree_Classifier = DecisionTreeClassifier(criterion="entropy", max_depth=100)

In [ ]:
"""
TRAINING TIME

"""

In [ ]:
# limit = 4000
# testing_subset = testing_data[:limit, :]
# training_subset = training_data[:limit, :]
# testing_sublabels = testing_labels[:limit]
# training_sublabels = training_labels[:limit ,]

solvers = ["Adam", "SGD"]

adam_runtime = [0.0]
adam_accuracy = [0.0]

sgd_runtime = [0.0]
sgd_accuracy = [0.0]

In [ ]:
for solver in solvers:
    NeuralNetwork_Classifier = MLPClassifier(solver=solver.lower(), max_iter=200, verbose=1, hidden_layer_sizes=(100,))
    for i in range(1, 11, 1):
        print("{} - Training Size: {}%".format(solver, (i * 10)))
        start_time = timer()
        with parallel_backend('threading'):
            NeuralNetwork_Classifier.fit(training_data[:int((60000 * (0.1 * i))), :], training_labels[:int((60000 * (0.1 * i)))])
        end_time = timer()
        elapsed_time = end_time - start_time
        print(elapsed_time)
        if solver == "SGD":
            sgd_accuracy.append(NeuralNetwork_Classifier.score(testing_data, testing_labels))
            sgd_runtime.append(elapsed_time)
        else:
            adam_accuracy.append(NeuralNetwork_Classifier.score(testing_data, testing_labels))
            adam_runtime.append(elapsed_time)

In [ ]:
adam_accuracy = np.asarray(adam_accuracy)
adam_runtime = np.asarray(adam_runtime)
sgd_accuracy = np.asarray(sgd_accuracy)
sgd_runtime = np.asarray(sgd_runtime)

In [ ]:
adam_accuracy.tofile('adam_accuracy_MNIST.csv',sep=',',format='%.3f')
adam_runtime.tofile('adam_runtime_MNIST.csv',sep=',',format='%.3f')
sgd_accuracy.tofile('sgd_accuracy_MNIST.csv',sep=',',format='%.3f')
sgd_runtime.tofile('sgd_runtime_MNIST.csv',sep=',',format='%.3f')

In [ ]:
with parallel_backend('threading'):
    KNN_Classifier.fit(training_subset, training_sublabels)

In [ ]:
with parallel_backend('threading'):
    Boosted_Classifier.fit(training_subset, training_sublabels)

In [ ]:
with parallel_backend('threading'):
    SVM_Classifier.fit(training_data, training_labels)

In [ ]:
with parallel_backend('threading'):
    DecisionTree_Classifier.fit(training_data, training_labels)

In [ ]:
"""
PREDICTIONS

"""

In [ ]:
plt.close('all')
Neural_Predict = NeuralNetwork_Classifier.predict(testing_data)
Neural_Report = metrics.classification_report(testing_labels, y_pred=Neural_Predict)
print(Neural_Report)

nn_confusion_matrix = metrics.plot_confusion_matrix(NeuralNetwork_Classifier, testing_data,
                                                 testing_labels, values_format=".4g")
nn_confusion_matrix.figure_.suptitle("Neural Network Confusion Matrix")
plt.show()

In [ ]:
plt.close('all')
KNN_Predict = KNN_Classifier.predict(testing_subset)
KNN_Report = metrics.classification_report(testing_sublabels, y_pred=KNN_Predict)
print(KNN_Report)

knn_confusion_matrix = metrics.plot_confusion_matrix(KNN_Classifier, testing_subset,
                                                 testing_sublabels, values_format=".4g")
knn_confusion_matrix.figure_.suptitle("K Nearest Neighbor Confusion Matrix")
plt.show()

In [ ]:
plt.close('all')
Boosted_Predict = Boosted_Classifier.predict(testing_subset)
Boosted_Report = metrics.classification_report(testing_sublabels, y_pred=Boosted_Predict)
print(Boosted_Report)

dt_confusion_matrix = metrics.plot_confusion_matrix(DecisionTree_Classifier, testing_subset,
                                                 testing_sublabels, values_format=".4g")
dt_confusion_matrix.figure_.suptitle("Boosted Decision Tree Confusion Matrix")
plt.show()

In [ ]:
plt.close('all')
SVM_Predict = SVM_Classifier.predict(testing_data)
SVM_Report = metrics.classification_report(testing_labels, y_pred=SVM_Predict)
print(SVM_Report)

svm_confusion_matrix = metrics.plot_confusion_matrix(SVM_Classifier, testing_data,
                                                 testing_labels, values_format=".4g")
svm_confusion_matrix.figure_.suptitle("Support Vector Machine Confusion Matrix")
plt.show()

In [ ]:
plt.close('all')
DecisionTree_Predict = DecisionTree_Classifier.predict(testing_data)
DecisionTree_Report = metrics.classification_report(testing_labels, y_pred=DecisionTree_Predict)
print(DecisionTree_Report)

dt_confusion_matrix = metrics.plot_confusion_matrix(DecisionTree_Classifier, testing_data,
                                                 testing_labels, values_format=".4g")
dt_confusion_matrix.figure_.suptitle("Decision Tree Confusion Matrix")
plt.show()

In [ ]:
DecisionTree_Classifier.get_depth()
# prune_index11(clf=DecisionTree_Classifier, index=0, threshold=50)
# print("Pruned")
# DecisionTree_Classifier.get_depth()

In [ ]:
print("Pruned")

In [ ]:
t = DecisionTree_Classifier
print(type(t))

In [ ]:
"""
Results

"""

In [ ]:
nn_results = FindCorrectPredictions(Neural_Predict, testing_labels)
print(nn_results)

In [ ]:
knn_results = FindCorrectPredictions(KNN_Predict, testing_sublabels)
print(nn_results)

In [ ]:
boosted_results = FindCorrectPredictions(Boosted_Predict,testing_sublabels)
print(boosted_results)

In [ ]:
svm_results = FindCorrectPredictions(SVM_Predict, testing_sublabels)
print(svm_results)

In [ ]:
dt_results = FindCorrectPredictions(DecisionTree_Predict, testing_labels)
print(dt_results)

In [ ]:
adam_accuracy = np.loadtxt("{}/adam_accuracy_MNIST.csv".format(cwd), delimiter=",")
adam_runtime = np.loadtxt("{}/adam_runtime_MNIST.csv".format(cwd), delimiter=",")

In [ ]:
sgd_accuracy = np.loadtxt("{}/sgd_accuracy_MNIST.csv".format(cwd), delimiter=",")
sgd_runtime = np.loadtxt("{}/sgd_runtime_MNIST.csv".format(cwd), delimiter=",")

In [ ]:
print(len(adam_accuracy))
print(adam_accuracy)
print(len(adam_runtime))
print(adam_runtime)
print(len(sgd_accuracy))
print(sgd_accuracy)
print(len(sgd_runtime))
print(sgd_runtime)

In [ ]:
print(solvers)

In [ ]:
colors = ["tab:orange", "tab:blue", "tab:green", "tab:red"]

run = [sgd_runtime, adam_runtime]
acc = [sgd_accuracy, adam_accuracy]

for solver in range(len(solvers)):
    with plt.style.context('ggplot'):
        fig0, ax0 = plt.subplots()
        ax0.set_xlabel("Percent of Training Set")
        ax0.set_ylabel("Accuracy (%)", color='tab:orange')
        ax0.set_title("Accuracy vs Training Set Size vs Training Time {} \n MNIST".format(solvers[solver]))
        ax0.tick_params(axis='y', labelcolor="black")
        ax0.set_ylim(0, 1.1)
        ax3 = ax0.twinx()
        ax3.set_ylabel("Training Time (s)", color="tab:blue")
        ax3.set_ylim(0, max(max(adam_runtime), max(sgd_runtime)) + 10)
        ax3.tick_params(axis='y', labelcolor="black")
        for i in range(1):        
            ax0.plot([i for i in range(11)], acc[solver], colors[i], marker='o', label=solvers[solver])
            ax3.plot([i for i in range(11)], run[solver], colors[i+1], marker="1", label="{} training-time".format(solvers[solver]))
        fig0.tight_layout()
        dir = "{}/Training_{}_MNIST_Set_Size_Impact_vs_Training_Time.png".format(cwd, solvers[solver])
        plt.savefig(dir)
#         plt.close("all")